# Functions For Coding Text Features

Below is a list of functions we developed to code for features to be used as predictors in our ML model.  

In [22]:
import random
import pandas as pd
import numpy as np
import scipy as sp
import nltk
import PyPDF2

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

## Past Tense Words

The function below tags each text answer for the presence of past tense words as classified by the nltk package.  Past tense is one of the many indices used in classic LIWC studies (e.g., Pennebaker & Francis, 1996/1999), which have been shown to significantly predict personality traits.  We cannot use the LIWC library here as it is not open access, but we can approximate it using the nltk package.  This function returns the percentage of past tense words in each answer.

In [5]:
def pastTag(user_input):
    past = 0
    words = user_input.split()
    for word in nltk.pos_tag(words):
        if (word[1] == "VBD") or (word[1] == "VBN"):
            past += 1
        percentpast = past/len(words)
    return percentpast  
         

## Present Tense Words

The presentTag() function tags each text answer for the presence (0) or absence (1) of present tense words as classified by the nltk package.  Is is also an approximation to the LIWC measure of present tense.  Fewer present tense words have been found to be predictive of openness to experience.

In [2]:
def presentTag(user_input):
    present = 0
     
    words = user_input.split()
    for word in nltk.pos_tag(words):
        if (word[1] == "VBG") or (word[1] == "VBP") or (word[1] == "VPZ"):
            present += 1
    percentpresent = present/len(words)
    return percentpresent

## Articles

The use of fewer articles (i.e., "a" or "the") has been found to positively predict agreeableness.  The following function caculates the percentage of articles in each text answer.

In [13]:
def articleTag(user_input):
    
    articles = 0
    words = user_input.split()
    for word in words:
        if (word == "a") or (word == "the") or (word == "A") or (word == "The"):
            articles += 1

    percentarticles = articles/len(words)
    return percentarticles


0.4

## First Person Singular

The use of more first person singular pronouns (i.e., "I", "me", "my" or "mine") has been shown to positively predict neuroticism and negatively predict openness to experience.  The following function calculates the percentage of first person singular pronouns used in each text answer.

In [16]:
def firstpersonTag(user_input):
    firstperson = 0
    words = user_input.split()
    for word in words:
        if (word == "I") or (word == "me") or (word == "my") or (word == "mine") or (word == "Me") or  (word == "My") or (word == "Mine"):
            firstperson += 1
        
    percentfirst = firstperson/len(words)
    return percentfirst
  

## Word Length

The wordLengths function measures the average word length of each text answer and then returns the percentage of words that have five or more letters.  Having a higher percentage of words greater than five letters long has been found to be a good predictor of openness to experience.

In [1]:
def wordLengths(user_input):
  
    words = list(map(len,user_input.split()))
    sumlengths = sum(i > 5 for i in words)
    percentoverfive = sumlengths/len(words)
    return percentoverfive

## Negative Affect Words

Negative affect is another index known to predict traits such as neuroticism (positively) and extraversion, agreeableness, and conscientiousness (negatively). We chose to use a publicly available libraries from Loughran-McDonald available here: https://sraf.nd.edu/textual-analysis/resources/#LM%20Sentiment%20Word%20Lists or here https://github.com/lcdm-uiuc/ml-finance.  The following syntax reads in the word list from a downloaded pdf and converts it into a vector that can be used to tag the text answers for the presence of negative words.

In [92]:
file = open('LM_Negative.pdf', 'rb')

# creating a pdf reader object
fileReader = PyPDF2.PdfFileReader(file)
pageObj = fileReader.getPage(0)
page_content = pageObj.extractText()

In [90]:
negativewords = ""
for page in range(fileReader.numPages):
    pageObj = fileReader.getPage(page)
    page_content = pageObj.extractText()
    negativewords = negativewords + page_content
negwords = negativewords.split()

negwords = negwords[7:]

The following function takes the list of words imported above (ie., "negwords") and tags each text answer as to the percentage of negative words present in its text.  

In [79]:
def negAffect(user_input):
    negative = 0
    words = user_input.split()
    for word in words:
        if word.upper() in negwords:
            negative = negative + 1

    percentnegative = negative/len(words)
    return percentnegative

## Positive Affect Words

We also used the Loughran-McDonald sentiment word list again (this time from https://github.com/lcdm-uiuc/ml-finance) to code for positive words.  Percentage of positive words has been found to positively predict extraversion, conscientiousness, and agreeableness and to negatively predict neuroticism.

In [101]:
file = open('LM_Positive.pdf', 'rb')

# creating a pdf reader object
fileReader = PyPDF2.PdfFileReader(file)
pageObj = fileReader.getPage(0)
page_content = pageObj.extractText()
#print(page_content)

In [103]:
positivewords = ""
for page in range(fileReader.numPages):
    pageObj = fileReader.getPage(page)
    page_content = pageObj.extractText()
    positivewords = positivewords + page_content
#print(negativewords)
poswords = positivewords.split()

poswords = poswords[7:]

In [104]:
def posAffect(user_input):
    positive = 0
    words = user_input.split()
    for word in words:
        if word.upper() in poswords:
            positive = positive + 1
        
    percentpositive = positive/len(words)
    return percentpositive

## Tentative Words

The use of tentative words that lessen certainty about a statement (e.g., "may", "possibly", "seems to") has been found to positively predict openness to experience and negatively predict extraversion.  We created a quick and dirty library of tentative words by looking online (mainly here: https://lo.unisa.edu.au/pluginfile.php/499128/mod_resource/content/2/Tentative%20language%20Nov%202015.pdf) to calculate the percentage of tentative words in each text answer.

In [18]:
tentwords = ["may", "might", "can", "could", "possibly", "probably", "likely", "possible", "unlikely", "probable", "tends", "appears", "suggests", "seems"]

In [19]:
def tentAffect(user_input):
    tentative = 0
    words = user_input.split()
    for word in words:
        if word.upper() in tentwords:
            tentative = tentative + 1
            print
       
    percenttentative = tentative/len(words)
    return percenttentative

tentAffect("I may go")

0.0

## Causal Words

Although more complicated text mining is usually needed to fully detect causal speech, we used a crude list of causal words from this publication (https://www.aaai.org/Papers/FLAIRS/2002/FLAIRS02-071.pdf) to do a quick tagging of causal speech as well.  Decreased use of causal words is usually positively associated with extraversion, conscientiousness, and openness to experience.

In [118]:
causewords = ["induce", "produce", "generate", "effect", "affect", "bring", "arouse", "provoke", "elicit", "lead", "trigger", 
             "derive", "associate", "relate", "link", "stem", "originate", "result", "stir", "entail", "contribute", "set", "commence",
             "conduce", "educe", "spark", "evoke", "implicate", "activate", "actuate", "kindle", "fire", "stimulate", "call", "unleash",
             "effectuate", "kick", "give", "birth", "call", "put", "create", "launch", "develop", "start", "make", "begin", "rise"]

In [6]:
def causeAffect(user_input):
    causal = 0
    words = user_input.split()
    for word in words:
        if word.upper() in causewords:
            causal = causal + 1
        
    percentcausal = causal/len(words)
    return percentcausal